In [ ]:
import glob
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pickle
import sys

# From arm
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.optim as optim
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from sklearn.linear_model import Lasso, LassoCV, LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm

from lifting_transformer.lifting_transformer import (
    criterion,
    data,
    helper,
    inference,
    model,
    training,
)
from mausspaun.data_processing.dlc import (
    DLC_TO_MUJOCO_MAPPING,
    MUJOCO_TO_DLC_MAPPING,
    align_data_with_rig_markers,
)
from mausspaun.data_processing.lifting.rig import MouseReachingRig5_2018
from mausspaun.data_processing.lifting.utils import _f32, _f64
from mausspaun.visualization.gui import app
from mausspaun.visualization.plot_3D_video import plot_3d_video, plot_split_3d_video

plt.style.use('cyhsm')

%load_ext autoreload
%autoreload 2

In [ ]:
def get_rig(calibration_folder="/data/markus/mausspaun/calibration/data/2018_mouse_reaching_rig5/calibration/"):
    return MouseReachingRig5_2018(calibration_folder)


def project_both_cameras(rig, points_3D):
    points_cam1 = project_points(points_3D, _f32(rig.stereo_rig[0].intrinsics), _f64(rig.stereo_rig[0].rotation),
                                 _f32(rig.stereo_rig[0].translation))
    points_cam2 = project_points(points_3D, _f32(rig.stereo_rig[1].intrinsics), _f64(rig.stereo_rig[1].rotation),
                                 _f32(rig.stereo_rig[1].translation))
    return (points_cam1, points_cam2)


def project_from_3D_dlc(inference_dlc_np):
    all_cam1, all_cam2 = [], []
    for i in range(0, inference_dlc_np.shape[1]):
        (cam1, cam2) = project_both_cameras(inference_dlc_np[:, i:i + 1, :])
        all_cam1.append(cam1)
        all_cam2.append(cam2)
    all_cam1 = np.transpose(np.array(all_cam1), axes=(2, 0, 1))
    all_cam2 = np.transpose(np.array(all_cam2), axes=(2, 0, 1))
    return (all_cam1, all_cam2)


def project_points(points_3D, intrinsic_matrix, rotation_matrix, translation_vector):
    p, _ = cv2.projectPoints(
        _f32(points_3D),
        rotation_matrix,
        translation_vector,
        intrinsic_matrix,
        distCoeffs=None,
    )
    return p.squeeze().T


def calculate_euclidean_error(array1, array2):
    squared_diff = np.sum(np.square(array1 - array2), axis=-1)
    return np.sqrt(squared_diff)

In [ ]:
mouse_name = 'HoneyBee'
day = 77
attempt = 1
part = 0
base_path = '/data/mausspaun/'

In [ ]:
# Test inference
(X_3d_test, X_2d_c1_test, X_2d_c2_test, cam_positions, likelihood_c1, likelihood_c2) = helper.load_single_session()
cam_positions_dict = {key: X_2d_c1_test[:, i, :] for i, key in enumerate(helper.mausspaun_keys)}
inference_preds = inference.run_inference(cam_positions_dict)

In [ ]:
avg_error, errors = inference.evaluate_ground_truth(inference_preds, mouse_name, day, attempt, verbose=2)

---
# Test triangulated 3D without using the transformer

In [ ]:
# Original triangulated:
filepaths = app.generate_filepaths(base_path + 'videos/videos_dlc2/', mouse_name, day, attempt, part)
X_3d_train, X_2d_c1_train, X_2d_c2_train, cam_positions, likelihood_c1, likelihood_c2 = helper.get_training_data(
    filepaths[0][1], filepaths[1][1], likelihood_cutoff=0, convert_to_mujoco=False)
X_3d_train_dict = {key: X_3d_train[:, i, :] for i, key in enumerate(helper.mausspaun_keys)}
X_3d_train_mujoco, (dlc_c, dlc_s, T, mujoco_c, mujoco_s) = helper.dlc_to_mujoco(X_3d_train_dict.copy())

In [ ]:
avg_error, errors = inference.evaluate_ground_truth(X_3d_train_mujoco, mouse_name, day, attempt, verbose=2)

---

In [ ]:
triangulated_3D = np.transpose(np.array([cp for key, cp in X_3d_train_mujoco.copy().items()]), axes=(1, 0, 2))
triangulated_3D_dlc = np.transpose(np.array([cp for key, cp in X_3d_train_dict.copy().items()]), axes=(1, 0, 2))

transformer_3D = np.transpose(np.array([cp for key, cp in inference_preds.copy().items()]), axes=(1, 0, 2))
original_2D_cam1 = X_2d_c1_train.copy()
original_2D_cam2 = X_2d_c2_train.copy()

print(
    f"Triangulated: {triangulated_3D.shape}, Transformer: {transformer_3D.shape}, 2D Cam1: {original_2D_cam1.shape}, 2D Cam2: {original_2D_cam2.shape}"
)
print(
    f"Triangulated Max: {np.max(triangulated_3D, axis=(0,1))}, Triangulated Min: {np.min(triangulated_3D, axis=(0,1))}")
print(
    f"Triangulated PctHigh: {np.percentile(triangulated_3D, 95, axis=(0,1))}, Triangulated PctLow: {np.percentile(triangulated_3D, 5, axis=(0,1))}"
)
print("---")
print(
    f"Triangulated Max: {np.max(triangulated_3D_dlc, axis=(0,1))}, Triangulated Min: {np.min(triangulated_3D_dlc, axis=(0,1))}"
)
print(
    f"Triangulated PctHigh: {np.percentile(triangulated_3D_dlc, 95, axis=(0,1))}, Triangulated PctLow: {np.percentile(triangulated_3D_dlc, 5, axis=(0,1))}"
)
print("---")
print(f"Transformer Max: {np.max(transformer_3D, axis=(0,1))}, Transformer Min: {np.min(transformer_3D, axis=(0,1))}")
print(f"2D Cam1 Max: {np.max(original_2D_cam1, axis=(0,1))}, 2D Cam1 Min: {np.min(original_2D_cam1, axis=(0,1))}")
print(
    f"2D Cam1 PctHigh: {np.percentile(original_2D_cam1, 95, axis=(0,1))}, 2D Cam1 PctLow: {np.percentile(original_2D_cam1, 5, axis=(0,1))}"
)
print(f"2D Cam2 Max: {np.max(original_2D_cam2, axis=(0,1))}, 2D Cam2 Min: {np.min(original_2D_cam2, axis=(0,1))}")
print(
    f"2D Cam2 PctHigh: {np.percentile(original_2D_cam2, 95, axis=(0,1))}, 2D Cam2 PctLow: {np.percentile(original_2D_cam2, 5, axis=(0,1))}"
)

In [ ]:
transformer_3D_dlc = helper.mujoco_to_dlc(inference_preds.copy(), dlc_c, dlc_s, T, mujoco_c, mujoco_s)
transformer_3D_dlc = np.transpose(np.array([cp for key, cp in transformer_3D_dlc.items()]), axes=(1, 0, 2))
print(
    f"Transformer DLC Max: {np.max(transformer_3D_dlc, axis=(0,1))}, Transformer DLC Min: {np.min(transformer_3D_dlc, axis=(0,1))}"
)
print(
    f"Transformer DLC PctHigh: {np.percentile(transformer_3D_dlc, 95, axis=(0,1))}, Transformer DLC PctLow: {np.percentile(transformer_3D_dlc, 5, axis=(0,1))}"
)

(all_cam1, all_cam2) = project_from_3D_dlc(transformer_3D_dlc)
print(f"Cam1 Max: {np.max(all_cam1, axis=(0,1))}, Cam1 Min: {np.min(all_cam1, axis=(0,1))}")
print(f"Cam1 PctHigh: {np.percentile(all_cam1, 95, axis=(0,1))}, Cam1 PctLow: {np.percentile(all_cam1, 5, axis=(0,1))}")

print(f"Cam2 Max: {np.max(all_cam2, axis=(0,1))}, Cam2 Min: {np.min(all_cam2, axis=(0,1))}")
print(f"Cam2 PctHigh: {np.percentile(all_cam2, 95, axis=(0,1))}, Cam2 PctLow: {np.percentile(all_cam2, 5, axis=(0,1))}")

In [ ]:
print(
    f"Triangulated DLC Max: {np.max(triangulated_3D_dlc, axis=(0,1))}, Triangulated DLC Min: {np.min(triangulated_3D_dlc, axis=(0,1))}"
)
print(
    f"Triangulated DLC PctHigh: {np.percentile(triangulated_3D_dlc, 95, axis=(0,1))}, Triangulated DLC PctLow: {np.percentile(triangulated_3D_dlc, 5, axis=(0,1))}"
)

(all_cam1, all_cam2) = project_from_3D_dlc(triangulated_3D_dlc)
print(f"Cam1 Max: {np.max(all_cam1, axis=(0,1))}, Cam1 Min: {np.min(all_cam1, axis=(0,1))}")
print(f"Cam1 PctHigh: {np.percentile(all_cam1, 95, axis=(0,1))}, Cam1 PctLow: {np.percentile(all_cam1, 5, axis=(0,1))}")

print(f"Cam2 Max: {np.max(all_cam2, axis=(0,1))}, Cam2 Min: {np.min(all_cam2, axis=(0,1))}")
print(f"Cam2 PctHigh: {np.percentile(all_cam2, 95, axis=(0,1))}, Cam2 PctLow: {np.percentile(all_cam2, 5, axis=(0,1))}")

---
# Project back to DLC space, then calculate reprojection error

In [ ]:
inference_dlc = helper.mujoco_to_dlc(inference_preds.copy(), dlc_c, dlc_s, T, mujoco_c, mujoco_s)
inference_dlc_np = np.transpose(np.array([cp for key, cp in inference_dlc.items()]), axes=(1, 0, 2))

(all_cam1_transformer, all_cam2_transformer) = project_from_3D_dlc(inference_dlc_np)
(all_cam1_triangulated, all_cam2_triangulated) = project_from_3D_dlc(triangulated_3D_dlc)

errors_transformer_cam1 = calculate_euclidean_error(X_2d_c1_train, all_cam1_transformer)
errors_transformer_cam2 = calculate_euclidean_error(X_2d_c2_train, all_cam2_transformer)
errors_triangulated_cam1 = calculate_euclidean_error(X_2d_c1_train, all_cam1_triangulated)
errors_triangulated_cam2 = calculate_euclidean_error(X_2d_c2_train, all_cam2_triangulated)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 5), sharex=True, sharey=True)
axes = axes.flatten()

for idx, (errors, txt) in enumerate(
        zip([errors_transformer_cam1, errors_transformer_cam2, errors_triangulated_cam1, errors_triangulated_cam2],
            ['Transformer Cam1', 'Transformer Cam2', 'Triangulated Cam1', 'Triangulated Cam2'])):
    axes[idx].matshow(np.log(errors.T), aspect='auto')
    axes[idx].set_title(txt)
axes[0].set_ylabel('Joints')
axes[2].set_xlabel('Time')
fig.tight_layout()